In [ ]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357268 sha256=a39f8f9e9d82ae48c74946735ca0397d5d4a8a0117632eae3d6b599b8285b29d
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [ ]:
import pandas as pd
import numpy as np
import surprise
import os

In [ ]:
os.chdir("/content/drive/MyDrive/Cases/ml-100k")
ratings = pd.read_csv("u.data",sep='\t',names = ['uid','iid','rating','ts'])
ratings.drop('ts', axis=1, inplace=True)
ratings.head()

,uid,iid,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [ ]:
lowest_rating = ratings['rating'].min()
highest_rating = ratings['rating'].max()
print("Ratings range between {0} and {1}".format(lowest_rating,highest_rating))

Ratings range between 1 and 5


In [ ]:
reader = surprise.Reader(rating_scale = (lowest_rating,highest_rating))
data = surprise.Dataset.load_from_df(ratings,reader)
type(data)

surprise.dataset.DatasetAutoFolds

Tuning for best parameters

In [ ]:
from surprise.model_selection import GridSearchCV
from surprise.model_selection.split import KFold

param_grid = {'lr_all':np.linspace(0.001,1,3), 'reg_all': np.linspace(0.01,0.8,3),
              'n_factors':[40,30]}
kfold = KFold(random_state=23,n_splits=5,shuffle=True)
gs = GridSearchCV(surprise.SVD, param_grid,joblib_verbose =3, measures=['rmse', 'mae'], cv=kfold,n_jobs=-1)
gs.fit(data)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:   30.7s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:  1.3min finished


Best Score:

In [ ]:
print(gs.best_score['rmse'])

0.9606990085224197


Best Parameter:

In [ ]:
print(gs.best_params['rmse'])

{'lr_all': 0.001, 'reg_all': 0.01, 'n_factors': 30}


We can now use the algorithm that yields the best rmse:

In [ ]:
algo = gs.best_estimator['rmse']
algo.fit(data.build_full_trainset())

The recommendations can be generated for any user with the object **algo**.

Total Items:

In [ ]:
iids = ratings['iid'].unique()
print(iids)

[ 242  302  377 ... 1637 1630 1641]


The list of items rated by user 50:

In [ ]:
u_iid = ratings[ratings['uid']==50]['iid'].unique()
print(u_iid)

[ 246  823  253  475 1084  286    9  125  123  325  508  288  319  324
  276 1008 1010  268  544   15  327  124  547  100]


List of the items not rated by user 50:

In [ ]:
iids_to_predict = np.setdiff1d(iids, u_iid)
print(iids_to_predict)

[   1    2    3 ... 1680 1681 1682]


Extracting the estimated rating from iids_to_predict

In [ ]:
testset = [[50,iid,0.] for iid in iids_to_predict]
predictions = algo.test(testset)

In [ ]:
predictions[0]

Prediction(uid=50, iid=1, r_ui=0.0, est=3.898337106926028, details={'was_impossible': False})

In [ ]:
pred_ratings = np.array([pred.est for pred in predictions])

Getting the item with highest expected rating

In [ ]:
i_max = np.argmax(pred_ratings)
print("Item:",iids_to_predict[i_max],"is the item with highest expected rating as",pred_ratings[i_max])

Item: 318 is the item with highest expected rating as 4.549750555889537


In [ ]:
exp_ratings = pd.DataFrame({'Item_ID': iids_to_predict, 'Exp_Rating':pred_ratings})
exp_ratings.sort_values(by=['Exp_Rating','Item_ID'], ascending=[False, True], inplace=True)
movies = pd.read_csv("/content/movies_list.csv", encoding='latin-1')
movies = movies[['movie id ',' movie title ']].rename(columns={'movie id ':'Item_ID'})
m_names = movies.merge(exp_ratings, left_on='Item_ID', right_on='Item_ID').sort_values(by=['Exp_Rating','Item_ID'], ascending=[False, True])
m_names.head(5)

,Item_ID,movie title,Exp_Rating
305,318,Schindler's List (1993),4.549751
465,483,Casablanca (1942),4.527147
391,408,"Close Shave, A (1995)",4.491659
61,64,"Shawshank Redemption, The (1994)",4.448209
10,12,"Usual Suspects, The (1995)",4.426867


In [ ]:
movies = pd.read_csv("/content/movies_list.csv", encoding='latin-1')
movies = movies[['movie id ',' movie title ']].rename(columns={'movie id ':'Item_ID'})
movies.merge(exp_ratings, left_on='Item_ID', right_on='Item_ID').sort_values(by=['Exp_Rating','Item_ID'], ascending=[False, True])


,Item_ID,movie title,Exp_Rating
305,318,Schindler's List (1993),4.549751
465,483,Casablanca (1942),4.527147
391,408,"Close Shave, A (1995)",4.491659
61,64,"Shawshank Redemption, The (1994)",4.448209
10,12,"Usual Suspects, The (1995)",4.426867
...,...,...,...
909,931,"Island of Dr. Moreau, The (1996)",2.588374
667,688,Leave It to Beaver (1997),2.573938
666,687,McHale's Navy (1997),2.553665
966,988,"Beautician and the Beast, The (1997)",2.549184
